<a href="https://colab.research.google.com/github/qkrehdwo1144/json_to_yolov5.txt/blob/main/json_to_yolov5_txt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# -*- coding: utf-8 -*-
import cv2
import numpy as np
import json
import yaml
import os
from pathlib import Path
from google.colab import files
from google.colab.patches import cv2_imshow
from google.colab import drive

# 기존 함수들은 그대로 유지 (read_json_file, convert_bbox_to_yolo, create_yolo_yaml, create_yolo_txt, visualize_traffic_lights)
def read_json_file(json_path):
    """JSON 파일을 읽어서 데이터를 반환합니다."""
    with open(json_path, 'r', encoding='utf-8') as f:
        data = json.load(f)
    return data

def convert_bbox_to_yolo(box, image_width, image_height):
    """바운딩 박스 좌표를 YOLO 형식으로 변환합니다."""
    x1, y1, x2, y2 = map(int, box)

    # YOLO 형식: <class> <x_center> <y_center> <width> <height>
    x_center = (x1 + x2) / (2 * image_width)
    y_center = (y1 + y2) / (2 * image_height)
    width = (x2 - x1) / image_width
    height = (y2 - y1) / image_height

    return [x_center, y_center, width, height]

def create_yolo_yaml(dataset_path, class_names):
    yaml_content = {
        'path': dataset_path,
        'train': 'images/train',
        'val': 'images/val',
        'test': 'images/test',
        'nc': len(class_names),
        'names': class_names
    }
    with open('traffic_light_dataset.yaml', 'w', encoding='utf-8') as f:
        yaml.dump(yaml_content, f, sort_keys=False, allow_unicode=True)

def create_yolo_txt(image_path, annotations, output_dir):
    """YOLO 형식의 레이블 텍스트 파일을 생성합니다."""
    image = cv2.imread(image_path)
    height, width = image.shape[:2]

    # 이미지 파일명과 동일한 텍스트 파일 생성
    base_name = Path(image_path).stem
    txt_path = os.path.join(output_dir, f"{base_name}.txt")

    with open(txt_path, 'w', encoding='utf-8') as f:
        for anno in annotations:
            if anno.get('class') == 'traffic_light':
                # 신호등 상태에 따른 클래스 인덱스 결정
                attributes = anno['attribute'][0]
                class_idx = 0  # 기본값

                # 신호 상태에 따른 클래스 분류 (예시)
                if attributes['red'] == 'on':
                    class_idx = 0  # 적색
                elif attributes['green'] == 'on':
                    class_idx = 1  # 녹색
                elif attributes['yellow'] == 'on':
                    class_idx = 2  # 황색
                elif attributes['left_arrow'] == 'on':
                    class_idx = 3  # 좌회전

                # YOLO 형식으로 좌표 변환
                yolo_bbox = convert_bbox_to_yolo(anno['box'], width, height)

                # class_idx x_center y_center width height 형식으로 저장
                f.write(f"{class_idx} {' '.join(map(str, yolo_bbox))}\n")

def visualize_traffic_lights(image_path, annotations):
    image = cv2.imread(image_path)

    for anno in annotations:
        if anno.get('class') == 'traffic_light':
            box = anno['box']
            x1, y1, x2, y2 = map(int, box)

            attributes = anno['attribute'][0]
            status = []
            if attributes['red'] == 'on':
                status.append('RED')
            if attributes['yellow'] == 'on':
                status.append('YELLOW')
            if attributes['green'] == 'on':
                status.append('GREEN')
            if attributes['left_arrow'] == 'on':
                status.append('←')
            if attributes['left_arrow'] == 'on' and attributes['green'] == 'on':
                status.append('↑←')
            if attributes['others_arrow'] == 'on':
                status.append('↔')
            if attributes['x_light'] == 'on':
                status.append('X')

            status_text = '+'.join(status) if status else 'OFF'

            cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(image, status_text, (x1, y1-10),
                       cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

            light_type = anno.get('type', 'unknown')
            light_count = anno.get('light_count', '?')
            info_text = f"Type: {light_type} ({light_count})"
            cv2.putText(image, info_text, (x1, y2+20),
                       cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    return image

def main():
    # Google Drive 마운트
    drive.mount('/content/drive')

    # 경로 설정
    json_dir = '/content/drive/MyDrive/json'  # JSON 파일들이 있는 디렉토리
    image_dir = '/content/drive/MyDrive/jpg'  # 이미지 파일들이 있는 디렉토리
    output_dir = '/content/drive/MyDrive/labels'  # YOLO 형식 레이블 저장 디렉토리
    visualization_dir = '/content/drive/MyDrive/output_jpg'  # 시각화 결과 저장 디렉토리

    # 출력 디렉토리 생성
    os.makedirs(output_dir, exist_ok=True)
    os.makedirs(visualization_dir, exist_ok=True)

    # JSON 파일 리스트 가져오기
    json_files = [f for f in os.listdir(json_dir) if f.endswith('.json')]

    # YOLO 데이터셋 설정
    class_names = ['red_light', 'green_light', 'yellow_light', 'left_arrow', 'straight_left_arrow']
    dataset_path = './dataset'  # 데이터셋 루트 경로

    # YAML 파일 생성
    create_yolo_yaml(dataset_path, class_names)

    for json_file in json_files:
        base_name = os.path.splitext(json_file)[0]
        image_file = f"{base_name}.jpg"

        json_path = os.path.join(json_dir, json_file)
        image_path = os.path.join(image_dir, image_file)

        if os.path.exists(image_path):
            # JSON 파일 읽기
            data = read_json_file(json_path)
            traffic_lights = [anno for anno in data['annotation']
                             if anno['class'] == 'traffic_light']

            # YOLO 형식의 레이블 파일 생성
            create_yolo_txt(image_path, traffic_lights, output_dir)

            # 시각화 실행
            result_image = visualize_traffic_lights(image_path, traffic_lights)

            # 시각화 결과 저장
            result_path = os.path.join(visualization_dir, f"{base_name}_visualized.jpg")
            cv2.imwrite(result_path, result_image)

            print(f"처리 완료: {json_file} 및 {image_file}")
            print(f"시각화 결과: {result_path}")
        else:
            print(f"이미지 파일을 찾을 수 없습니다: {image_file}")

    print(f"YOLO 형식 레이블이 {output_dir} 디렉토리에 생성되었습니다.")
    print(f"시각화 결과가 {visualization_dir} 디렉토리에 저장되었습니다.")
    print("YAML 파일이 생성되었습니다: traffic_light_dataset.yaml")

if __name__ == "__main__":
    main()

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
시각화 결과: /content/drive/MyDrive/output_jpg/13856469_visualized.jpg
처리 완료: 13856390.json 및 13856390.jpg
시각화 결과: /content/drive/MyDrive/output_jpg/13856390_visualized.jpg
처리 완료: 13908935.json 및 13908935.jpg
시각화 결과: /content/drive/MyDrive/output_jpg/13908935_visualized.jpg
처리 완료: 13986576.json 및 13986576.jpg
시각화 결과: /content/drive/MyDrive/output_jpg/13986576_visualized.jpg
처리 완료: 13700642.json 및 13700642.jpg
시각화 결과: /content/drive/MyDrive/output_jpg/13700642_visualized.jpg
처리 완료: 13986590.json 및 13986590.jpg
시각화 결과: /content/drive/MyDrive/output_jpg/13986590_visualized.jpg
처리 완료: 13826969.json 및 13826969.jpg
시각화 결과: /content/drive/MyDrive/output_jpg/13826969_visualized.jpg
처리 완료: 13732878.json 및 13732878.jpg
시각화 결과: /content/drive/MyDrive/output_jpg/13732878_visualized.jpg
처리 완료: 13672932.json 및 13672932.jpg
시각화 결과: /content/drive/MyDrive/output_jpg/13672932_visualized.jpg
처리 완료: 13815217.json 및 13815217.jpg
시각화 결과: /content/drive/MyDrive/output_jpg/1381